
---

# **Notebook 10 : Traduction des Noms de Pays en Français**

---

### **1. Importation des Bibliothèques**

Dans cette section, nous importons les bibliothèques nécessaires pour la lecture des fichiers, la manipulation des données, et la traduction.



In [1]:
# Importation des bibliothèques
import os
import pandas as pd
from googletrans import Translator  # Assurez-vous d'avoir installé googletrans==4.0.0-rc1
import time



### **2. Définition de la fonction de regroupement et de traduction des pays**

Cette fonction lit les fichiers de chunks, extrait tous les noms de pays uniques, les traduit une seule fois, et applique les traductions sur chaque chunk.



In [2]:
def traduire_pays_en_francais(input_directory, output_directory):
    # Initialiser le traducteur
    translator = Translator()
    
    # Récupérer tous les noms de pays uniques
    pays_uniques = set()
    for file_name in os.listdir(input_directory):
        input_file_path = os.path.join(input_directory, file_name)
        chunk_iter = pd.read_csv(input_file_path, sep=',', chunksize=10000, low_memory=False, on_bad_lines="skip")
        
        for chunk in chunk_iter:
            if 'countries' in chunk.columns:
                # Ajouter les pays uniques dans l'ensemble
                chunk['countries_cleaned'] = chunk['countries'].str.split(',').apply(lambda x: [p.strip() for p in x if p.strip()])
                pays_uniques.update([p for sublist in chunk['countries_cleaned'] for p in sublist])

    # Traduire tous les pays uniques en français
    pays_traduits = {}
    for pays in pays_uniques:
        try:
            traduction = translator.translate(pays, src='auto', dest='fr').text
            pays_traduits[pays] = traduction
            time.sleep(0.2)  # Délai pour éviter de surcharger l'API
        except Exception as e:
            print(f"Erreur de traduction pour {pays}: {e}")
            pays_traduits[pays] = None  # Ajouter None pour les pays non traduits
    
    # Appliquer les traductions sur chaque fichier de chunk et sauvegarder les résultats
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    for file_name in os.listdir(input_directory):
        input_file_path = os.path.join(input_directory, file_name)
        output_file_path = os.path.join(output_directory, file_name)
        
        # Lire chaque chunk, traduire les pays, et enregistrer
        chunk_iter = pd.read_csv(input_file_path, sep=',', chunksize=10000, low_memory=False, on_bad_lines="skip")
        for i, chunk in enumerate(chunk_iter):
            if 'countries' in chunk.columns:
                # Appliquer les traductions de pays
                chunk['countries_cleaned'] = chunk['countries'].str.split(',').apply(lambda x: [p.strip() for p in x if p.strip()])
                chunk['countries_translated'] = chunk['countries_cleaned'].apply(
                    lambda country_list: [pays_traduits.get(country, country) for country in country_list if pays_traduits.get(country)]
                )
                
                # Enregistrer chaque chunk traduit
                chunk.to_csv(output_file_path, mode='a', index=False, header=(i == 0))
                
                print(f"Chunk {i+1} de {file_name} traité et enregistré.")


### **3. DDéfinition des répertoires d'entrée et de sortie**

Dans cette section, configurez les répertoires d'entrée et de sortie pour les chunks.


In [3]:
# Répertoires d'entrée et de sortie
input_directory = 'note-books/phase_1_data/07_chunk_no_redundant_columns'
output_directory = 'note-books/phase_1_data/09_translated_countries'


### **4. Exécution de la Fonction de Traduction**

Exécutez la fonction traduire_pays_en_francais pour appliquer les traductions et sauvegarder les résultats.

In [ ]:
# Exécuter la fonction
traduire_pays_en_francais(input_directory, output_directory)



---

### **Explications :**

- **Traduction par lots** : : Cette méthode extrait les noms de pays uniques et les traduit une seule fois, réduisant le nombre de requêtes API.
- **Application sur chaque chunk**  : Les traductions sont appliquées au niveau des chunks pour un traitement efficace.
- **Gestion des erreurs** : Les pays qui ne peuvent pas être traduits sont ignorés dans le résultat final.

---